# **Projek Akhir Sentimen Analisis pada Komentar Mengenai Mobile JKN dengan Menggunakan Algoritme Naïve Bayes**

##Importing Library

In [ ]:
from google.colab import files
import pandas as pd
import numpy as np
from collections import Counter
import re
import string
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
!pip install Sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
     |████████████████████████████████| 209 kB 14.2 MB/s 


##Importing the Dataset

In [ ]:
kontak = files.upload()
for fn in kontak.keys():
  print('Nama file "{name}" dengan panjang {length} bytes'.format(
      name=fn, length=len(kontak[fn])))

Saving dataset.csv to dataset.csv
Nama file "dataset.csv" dengan panjang 5888 bytes


In [ ]:
data = pd.read_csv('dataset.csv')

data.head(10)

,tweet,label
0,Sangat baik aplikasi ini dapat cetak kartu JKN...,1
1,Mempermudah Masyarakat hanya sekali klik tanpa...,1
2,"Makasih...sangat membantu banget aplikasi nya,",1
3,Cukup membantu pelayanan di klinik atau rumah ...,1
4,Mobile JKN sangat membangun sekali untuk mempe...,1
5,aplikasinya sangsat membantu untuk mengecek ak...,1
6,Bagus aplikasinya buat mengetahui tentang bpjs...,1
7,Dengan apkikasi baru yg ada fitur2 tambahan ko...,1
8,"mempermudah untuk cek pelayanan FKTP , terima ...",1
9,Sengat membantu sekali dengan ada nya aplikasi...,1


##Preprocessing

In [ ]:
def preprocessing(dokumen):
  dokumen = dokumen.lower()
  dokumen = re.sub("-"," ", dokumen)
  dokumen = [word.strip(string.punctuation) for word in dokumen.split(" ")]
  dokumen = [word for word in dokumen if not any(c.isdigit() for c in word)]
  stopword = stopwords.words('indonesian')
  dokumen = [x for x in dokumen if x not in stopword]
  factory = StemmerFactory()
  stemmer = factory.create_stemmer()
  dokumen = [stemmer.stem(word) for word in dokumen]
  dokumen = [t for t in dokumen if len(t) > 1]
  dokumen = " ".join(dokumen)
  return(dokumen)

In [ ]:
#Preprocessing Dataset
data['tweet'] = data['tweet'].apply(lambda x: preprocessing(x))
data.head(10)

,tweet,label
0,aplikasi cetak kartu jkn cepat,1
1,mudah masyarakat klik antri,1
2,makasih sangat bantu banget aplikasi nya,1
3,bantu layan klinik rumah sakit,1
4,mobile jkn bangun mudah sistem,1
5,aplikasi sangsat bantu ecek aktip nya pbjs sehat,1
6,bagus aplikasi bpjs bayar bulan dll,1
7,apkikasi yg tambah komsultasi dokter jadwal op...,1
8,mudah cek layan fktp terima kasih bpjs,1
9,sengat bantu nya aplikasi inih jd tau info tnt...,1


#Fase Training

## Memisahkan Data per Kelas

In [ ]:
# Data Kelas 1
data_positif = [row['tweet'] for index, row in data.iterrows() if row['label'] == 1]
data_positif

['aplikasi cetak kartu jkn cepat',
 'mudah masyarakat klik antri',
 'makasih sangat bantu banget aplikasi nya',
 'bantu layan klinik rumah sakit',
 'mobile jkn bangun mudah sistem',
 'aplikasi sangsat bantu ecek aktip nya pbjs sehat',
 'bagus aplikasi bpjs bayar bulan dll',
 'apkikasi yg tambah komsultasi dokter jadwal operasi bagus',
 'mudah cek layan fktp terima kasih bpjs',
 'sengat bantu nya aplikasi inih jd tau info tntng bpjs',
 'mudah cari informasi jkn',
 'terima kasih mobile jkn layan nya',
 'oke si bantu tp updat trus udah',
 'bantu fasilitas sehat',
 'aplikasi bantu tingkat']

In [ ]:
data_negatif = [row['tweet'] for index, row in data.iterrows() if row['label'] == 0]
data_negatif

['jkn anjiiing kali hubung call centernya daftar serta gak pulsa si tarik habis hasil nihil mudah sulit orang daftar bpjs nelfon pulsa tarik barapa kali hubung k ulas lengkapjkn anjiiing kali hubung call centernya daftar serta gak pulsa si tarik habis hasil nihil mudah sulit orang daftar bpjs nelfon pulsa tarik barapa kali hubung call center daftar aplikasi gak mo daftar anggota aja gak tolong tingkat layan susah rakyat mulu',
 'tolong jelas kmren daftar dibpjs online login jawab transakai proses silah coba sdh coba jawab bgitu klo mmg tdak kasih pasti jgn ulas lengkaptolong jelas kmren daftar dibpjs online login jawab transakai proses silah coba sdh coba jawab bgitu klo mmg tdak kasih pasti jgn bantu layan butuh tolong perintah henti berhoax mudah rakyat rakyat tetangga',
 'transaksi proses ulang tdk buka tolong baik apps nya yg complaint tp buruk jg nih apps',
 'aplikasi sulit akses kalo jam jam sibuk tampil transaksi proses silah coba obat kalo jam malam aplikasi lancar',
 'daaftar 

## Split Data Latih dan Data Uji

In [ ]:
def data_split(pertama, kedua, ketiga):
  data_uji=[]
  data_latih_p=[]
  data_latih_n=[]
  for i in range(0,15):
    if(i==pertama or i==kedua or i==ketiga):
      data_uji.append(data_positif[i])
      data_uji.append(data_negatif[i])
    else :
      data_latih_p.append(data_positif[i])
      data_latih_n.append(data_negatif[i])

  data = np.concatenate((data_uji, data_latih_p, data_latih_n))
  return data_uji, data_latih_p, data_latih_n

In [ ]:
data_split = data_split(0,1,2)

In [ ]:
#Data Uji
data_uji = data_split[0]
print(data_uji)

['aplikasi cetak kartu jkn cepat', 'jkn anjiiing kali hubung call centernya daftar serta gak pulsa si tarik habis hasil nihil mudah sulit orang daftar bpjs nelfon pulsa tarik barapa kali hubung k ulas lengkapjkn anjiiing kali hubung call centernya daftar serta gak pulsa si tarik habis hasil nihil mudah sulit orang daftar bpjs nelfon pulsa tarik barapa kali hubung call center daftar aplikasi gak mo daftar anggota aja gak tolong tingkat layan susah rakyat mulu', 'mudah masyarakat klik antri', 'tolong jelas kmren daftar dibpjs online login jawab transakai proses silah coba sdh coba jawab bgitu klo mmg tdak kasih pasti jgn ulas lengkaptolong jelas kmren daftar dibpjs online login jawab transakai proses silah coba sdh coba jawab bgitu klo mmg tdak kasih pasti jgn bantu layan butuh tolong perintah henti berhoax mudah rakyat rakyat tetangga', 'makasih sangat bantu banget aplikasi nya', 'transaksi proses ulang tdk buka tolong baik apps nya yg complaint tp buruk jg nih apps']


In [ ]:
#Data Latih
data_latih_p = data_split[1]
data_latih_n = data_split[2]

data_latih = np.concatenate((data_latih_p, data_latih_n))
print(data_latih)

['bantu layan klinik rumah sakit' 'mobile jkn bangun mudah sistem'
 'aplikasi sangsat bantu ecek aktip nya pbjs sehat'
 'bagus aplikasi bpjs bayar bulan dll'
 'apkikasi yg tambah komsultasi dokter jadwal operasi bagus'
 'mudah cek layan fktp terima kasih bpjs'
 'sengat bantu nya aplikasi inih jd tau info tntng bpjs'
 'mudah cari informasi jkn' 'terima kasih mobile jkn layan nya'
 'oke si bantu tp updat trus udah' 'bantu fasilitas sehat'
 'aplikasi bantu tingkat'
 'aplikasi sulit akses kalo jam jam sibuk tampil transaksi proses silah coba obat kalo jam malam aplikasi lancar'
 'daaftar aja susah udah download blg drmh aja aplikasi nya gak bener tolong nambah susahin daftar aja susah benrr gak'
 'knp ya bs bayar bpjs sehat virtual account yg pake bs bayar tera tulisa id no tlpn npwp maret bs byr'
 'layan buruk tiap buka aplikasi jkn suruh baharu play store saya bayar premi cek jkn tagih dan kartu non aktif premi'
 'daftar ajha ribet bgt kantor nya suruh aplikasi aplikasi gk komplain kanto

## Term Frequency

In [ ]:
#Term Frequency Keseluruhan
term = ' '.join([i for i in data_latih]).split()
tf = dict(Counter((" ".join(term).split(" "))))
tf

{'account': 1,
 'aja': 5,
 'ajha': 1,
 'akses': 3,
 'aktif': 1,
 'aktip': 1,
 'akun': 3,
 'alias': 1,
 'apk': 1,
 'apkikasi': 1,
 'aplikasi': 20,
 'arah': 2,
 'bagus': 3,
 'baharu': 1,
 'baik': 5,
 'bangun': 1,
 'bantu': 6,
 'bayar': 5,
 'bener': 2,
 'benerin': 1,
 'benrr': 1,
 'better': 1,
 'bgt': 1,
 'bicara': 1,
 'bikin': 1,
 'bkin': 2,
 'blg': 1,
 'blm': 1,
 'bpjs': 4,
 'bs': 3,
 'btw': 1,
 'buka': 4,
 'bulak': 1,
 'bulan': 1,
 'buruk': 1,
 'byr': 1,
 'call': 2,
 'capil': 1,
 'capta': 2,
 'captanya': 1,
 'cari': 1,
 'cek': 2,
 'center': 2,
 'coba': 4,
 'daaftar': 1,
 'daftar': 8,
 'dan': 1,
 'dear': 1,
 'development': 1,
 'diupdate': 1,
 'dll': 1,
 'dokter': 1,
 'donk': 1,
 'download': 1,
 'drmh': 1,
 'ecek': 1,
 'fasilitas': 1,
 'fitur': 1,
 'fktp': 1,
 'ga': 1,
 'gabisa': 1,
 'gagal': 2,
 'gak': 3,
 'gambar': 1,
 'gimna': 1,
 'gin': 1,
 'gk': 2,
 'hari': 1,
 'hubung': 2,
 'huft': 1,
 'id': 1,
 'in': 5,
 'info': 1,
 'informasi': 1,
 'inih': 1,
 'install': 1,
 'it': 1,
 'iya': 1,
 

In [ ]:
#Term Frequency Kelasi 1
tf_positif= dict(Counter((" ".join(data_latih_p).split(" "))))
tf_positif

{'aktip': 1,
 'apkikasi': 1,
 'aplikasi': 4,
 'bagus': 2,
 'bangun': 1,
 'bantu': 6,
 'bayar': 1,
 'bpjs': 3,
 'bulan': 1,
 'cari': 1,
 'cek': 1,
 'dll': 1,
 'dokter': 1,
 'ecek': 1,
 'fasilitas': 1,
 'fktp': 1,
 'info': 1,
 'informasi': 1,
 'inih': 1,
 'jadwal': 1,
 'jd': 1,
 'jkn': 3,
 'kasih': 2,
 'klinik': 1,
 'komsultasi': 1,
 'layan': 3,
 'mobile': 2,
 'mudah': 3,
 'nya': 3,
 'oke': 1,
 'operasi': 1,
 'pbjs': 1,
 'rumah': 1,
 'sakit': 1,
 'sangsat': 1,
 'sehat': 2,
 'sengat': 1,
 'si': 1,
 'sistem': 1,
 'tambah': 1,
 'tau': 1,
 'terima': 2,
 'tingkat': 1,
 'tntng': 1,
 'tp': 1,
 'trus': 1,
 'udah': 1,
 'updat': 1,
 'yg': 1}

In [ ]:
#Term Frequency Kelasi 0
tf_negatif = dict(Counter((" ".join(data_latih_n).split(" "))))
tf_negatif

{'account': 1,
 'aja': 5,
 'ajha': 1,
 'akses': 3,
 'aktif': 1,
 'akun': 3,
 'alias': 1,
 'apk': 1,
 'aplikasi': 16,
 'arah': 2,
 'bagus': 1,
 'baharu': 1,
 'baik': 5,
 'bayar': 4,
 'bener': 2,
 'benerin': 1,
 'benrr': 1,
 'better': 1,
 'bgt': 1,
 'bicara': 1,
 'bikin': 1,
 'bkin': 2,
 'blg': 1,
 'blm': 1,
 'bpjs': 1,
 'bs': 3,
 'btw': 1,
 'buka': 4,
 'bulak': 1,
 'buruk': 1,
 'byr': 1,
 'call': 2,
 'capil': 1,
 'capta': 2,
 'captanya': 1,
 'cek': 1,
 'center': 2,
 'coba': 4,
 'daaftar': 1,
 'daftar': 8,
 'dan': 1,
 'dear': 1,
 'development': 1,
 'diupdate': 1,
 'donk': 1,
 'download': 1,
 'drmh': 1,
 'fitur': 1,
 'ga': 1,
 'gabisa': 1,
 'gagal': 2,
 'gak': 3,
 'gambar': 1,
 'gimna': 1,
 'gin': 1,
 'gk': 2,
 'hari': 1,
 'hubung': 2,
 'huft': 1,
 'id': 1,
 'in': 5,
 'install': 1,
 'it': 1,
 'iya': 1,
 'jam': 5,
 'jgn': 1,
 'jkn': 2,
 'kalah': 1,
 'kali': 1,
 'kalo': 2,
 'kantor': 2,
 'kartu': 1,
 'kasih': 1,
 'kembang': 1,
 'kerja': 2,
 'kesel': 1,
 'kesiap': 2,
 'klik': 1,
 'kn': 1,
 '

## List Fitur

In [ ]:
# Semua Fitur
fitur = tf.keys()
fitur

dict_keys(['bantu', 'layan', 'klinik', 'rumah', 'sakit', 'mobile', 'jkn', 'bangun', 'mudah', 'sistem', 'aplikasi', 'sangsat', 'ecek', 'aktip', 'nya', 'pbjs', 'sehat', 'bagus', 'bpjs', 'bayar', 'bulan', 'dll', 'apkikasi', 'yg', 'tambah', 'komsultasi', 'dokter', 'jadwal', 'operasi', 'cek', 'fktp', 'terima', 'kasih', 'sengat', 'inih', 'jd', 'tau', 'info', 'tntng', 'cari', 'informasi', 'oke', 'si', 'tp', 'updat', 'trus', 'udah', 'fasilitas', 'tingkat', 'sulit', 'akses', 'kalo', 'jam', 'sibuk', 'tampil', 'transaksi', 'proses', 'silah', 'coba', 'obat', 'malam', 'lancar', 'daaftar', 'aja', 'susah', 'download', 'blg', 'drmh', 'gak', 'bener', 'tolong', 'nambah', 'susahin', 'daftar', 'benrr', 'knp', 'ya', 'bs', 'virtual', 'account', 'pake', 'tera', 'tulisa', 'id', 'no', 'tlpn', 'npwp', 'maret', 'byr', 'buruk', 'tiap', 'buka', 'suruh', 'baharu', 'play', 'store', 'saya', 'premi', 'tagih', 'dan', 'kartu', 'non', 'aktif', 'ajha', 'ribet', 'bgt', 'kantor', 'gk', 'komplain', 'nik', 'blm', 'online', 'k

In [ ]:
# Semua Fitur Kelas 1
fitur_positif = tf_positif.keys()
fitur_positif

dict_keys(['bantu', 'layan', 'klinik', 'rumah', 'sakit', 'mobile', 'jkn', 'bangun', 'mudah', 'sistem', 'aplikasi', 'sangsat', 'ecek', 'aktip', 'nya', 'pbjs', 'sehat', 'bagus', 'bpjs', 'bayar', 'bulan', 'dll', 'apkikasi', 'yg', 'tambah', 'komsultasi', 'dokter', 'jadwal', 'operasi', 'cek', 'fktp', 'terima', 'kasih', 'sengat', 'inih', 'jd', 'tau', 'info', 'tntng', 'cari', 'informasi', 'oke', 'si', 'tp', 'updat', 'trus', 'udah', 'fasilitas', 'tingkat'])

In [ ]:
# Semua Fitur Kelas 0
fitur_negatif = tf_negatif.keys()
fitur_negatif

dict_keys(['aplikasi', 'sulit', 'akses', 'kalo', 'jam', 'sibuk', 'tampil', 'transaksi', 'proses', 'silah', 'coba', 'obat', 'malam', 'lancar', 'daaftar', 'aja', 'susah', 'udah', 'download', 'blg', 'drmh', 'nya', 'gak', 'bener', 'tolong', 'nambah', 'susahin', 'daftar', 'benrr', 'knp', 'ya', 'bs', 'bayar', 'bpjs', 'sehat', 'virtual', 'account', 'yg', 'pake', 'tera', 'tulisa', 'id', 'no', 'tlpn', 'npwp', 'maret', 'byr', 'layan', 'buruk', 'tiap', 'buka', 'jkn', 'suruh', 'baharu', 'play', 'store', 'saya', 'premi', 'cek', 'tagih', 'dan', 'kartu', 'non', 'aktif', 'ajha', 'ribet', 'bgt', 'kantor', 'gk', 'komplain', 'nik', 'blm', 'online', 'kn', 'capil', 'huft', 'rempong', 'sih', 'meribetkan', 'gin', 'diupdate', 'gabisa', 'masuk', 'minggu', 'uninstall', 'install', 'ulang', 'oke', 'fitur', 'tambah', 'moga', 'bagus', 'capta', 'captanya', 'alias', 'kosong', 'gimna', 'dear', 'kembang', 'saran', 'mudah', 'mohon', 'baik', 'akun', 'pas', 'log', 'in', 'nomor', 'terang', 'sumpah', 'kesel', 'kali', 'bulak

## Conditional Probability Fitur

In [ ]:
def conditional_prob_positif(uji) :
  prob_p = []
  for i in uji:
    if i in tf_positif.keys():
      count =  tf_positif[i]
    else :
      count = 0
    prob_p.append((count+1)/(len(tf_positif)+len(term)))
  return dict(zip(uji,prob_p))

def conditional_prob_negatif(uji) :
  prob_n = []
  for i in uji:
    if i in tf_negatif.keys():
      count =  tf_negatif[i]
    else :
      count = 0
    prob_n.append((count+1)/(len(tf_negatif)+len(term)))
  return dict(zip(uji,prob_n))

#Fase Testing

### Menghitung Nilai Prior

In [ ]:
def prior(list_kelas):
  n_data = len(list_kelas)
  prior = Counter(list_kelas)
  for key in prior.keys():
    prior[key]=prior[key]/n_data
  return prior

In [ ]:
prior = prior(data['label'])
print(prior)

Counter({1: 0.5, 0: 0.5})


### Menampilkan nilai term data uji

In [ ]:
def term_uji(data_uji) :
  term_uji =  ' '.join([i for i in data_uji.split(" ")]).split()
  return term_uji

In [ ]:
term_data_uji = []
for i in range(0, len(data_uji)) :
  term_data_uji.append(term_uji(data_uji[i]))

term_data_uji

[['aplikasi', 'cetak', 'kartu', 'jkn', 'cepat'],
 ['jkn',
  'anjiiing',
  'kali',
  'hubung',
  'call',
  'centernya',
  'daftar',
  'serta',
  'gak',
  'pulsa',
  'si',
  'tarik',
  'habis',
  'hasil',
  'nihil',
  'mudah',
  'sulit',
  'orang',
  'daftar',
  'bpjs',
  'nelfon',
  'pulsa',
  'tarik',
  'barapa',
  'kali',
  'hubung',
  'k',
  'ulas',
  'lengkapjkn',
  'anjiiing',
  'kali',
  'hubung',
  'call',
  'centernya',
  'daftar',
  'serta',
  'gak',
  'pulsa',
  'si',
  'tarik',
  'habis',
  'hasil',
  'nihil',
  'mudah',
  'sulit',
  'orang',
  'daftar',
  'bpjs',
  'nelfon',
  'pulsa',
  'tarik',
  'barapa',
  'kali',
  'hubung',
  'call',
  'center',
  'daftar',
  'aplikasi',
  'gak',
  'mo',
  'daftar',
  'anggota',
  'aja',
  'gak',
  'tolong',
  'tingkat',
  'layan',
  'susah',
  'rakyat',
  'mulu'],
 ['mudah', 'masyarakat', 'klik', 'antri'],
 ['tolong',
  'jelas',
  'kmren',
  'daftar',
  'dibpjs',
  'online',
  'login',
  'jawab',
  'transakai',
  'proses',
  'silah',


### Menghitung Conditional Probrability Data Uji

In [ ]:
# Conditional Probrability Positif
cp_p = []
for i in range (0, 6) :
  cp_p.append(conditional_prob_positif(term_data_uji[i]))

In [ ]:
# Conditional Probrability Negatif
cp_n = []
for i in range (0, 6) :
  cp_n.append(conditional_prob_negatif(term_data_uji[i]))

In [ ]:
cp_p

[{'aplikasi': 0.012195121951219513,
  'cepat': 0.0024390243902439024,
  'cetak': 0.0024390243902439024,
  'jkn': 0.00975609756097561,
  'kartu': 0.0024390243902439024},
 {'aja': 0.0024390243902439024,
  'anggota': 0.0024390243902439024,
  'anjiiing': 0.0024390243902439024,
  'aplikasi': 0.012195121951219513,
  'barapa': 0.0024390243902439024,
  'bpjs': 0.00975609756097561,
  'call': 0.0024390243902439024,
  'center': 0.0024390243902439024,
  'centernya': 0.0024390243902439024,
  'daftar': 0.0024390243902439024,
  'gak': 0.0024390243902439024,
  'habis': 0.0024390243902439024,
  'hasil': 0.0024390243902439024,
  'hubung': 0.0024390243902439024,
  'jkn': 0.00975609756097561,
  'k': 0.0024390243902439024,
  'kali': 0.0024390243902439024,
  'layan': 0.00975609756097561,
  'lengkapjkn': 0.0024390243902439024,
  'mo': 0.0024390243902439024,
  'mudah': 0.00975609756097561,
  'mulu': 0.0024390243902439024,
  'nelfon': 0.0024390243902439024,
  'nihil': 0.0024390243902439024,
  'orang': 0.002439

In [ ]:
cp_n

[{'aplikasi': 0.032196969696969696,
  'cepat': 0.001893939393939394,
  'cetak': 0.001893939393939394,
  'jkn': 0.005681818181818182,
  'kartu': 0.003787878787878788},
 {'aja': 0.011363636363636364,
  'anggota': 0.001893939393939394,
  'anjiiing': 0.001893939393939394,
  'aplikasi': 0.032196969696969696,
  'barapa': 0.001893939393939394,
  'bpjs': 0.003787878787878788,
  'call': 0.005681818181818182,
  'center': 0.005681818181818182,
  'centernya': 0.001893939393939394,
  'daftar': 0.017045454545454544,
  'gak': 0.007575757575757576,
  'habis': 0.001893939393939394,
  'hasil': 0.001893939393939394,
  'hubung': 0.005681818181818182,
  'jkn': 0.005681818181818182,
  'k': 0.001893939393939394,
  'kali': 0.003787878787878788,
  'layan': 0.015151515151515152,
  'lengkapjkn': 0.001893939393939394,
  'mo': 0.001893939393939394,
  'mudah': 0.003787878787878788,
  'mulu': 0.003787878787878788,
  'nelfon': 0.001893939393939394,
  'nihil': 0.001893939393939394,
  'orang': 0.001893939393939394,
  '

### Menghitung Nilai Posterior

In [ ]:
def posterior(conditional_prob, prior) :
  peluang_p = 1
  for i in conditional_prob :
    peluang_p = peluang_p * conditional_prob[i]
  posterior = peluang_p * prior
  return posterior

In [ ]:
# Posterior Positif
posterior_p = []
for i in range (0, 6) :
  posterior_p.append(posterior(cp_p[i], prior[1]))

# Posterior Negatif
posterior_n = []
for i in range (0, 6) :
  posterior_n.append(posterior(cp_n[i], prior[0]))

In [ ]:
posterior_p

[8.631389527436687e-13,
 9.137028511914271e-89,
 7.077739412498085e-11,
 1.694375735381135e-79,
 1.4736518705379717e-14,
 5.144369280628751e-39]

### Menentukan Kelas Data Uji

In [ ]:
label_uji = []
for i in range(0,6) :
  if posterior_p[i] > posterior_n[i] :
    label_uji.append(1)
  elif posterior_p[i] < posterior_n[i]:
    label_uji.append(0)

In [ ]:
label_uji

[0, 0, 1, 0, 1, 0]

### Menetukan Nilai Akurasi Prediksi

In [ ]:
def func_prediksi(label_uji):
  data_uji_label = [1, 0, 1, 0, 1, 0]
  prediksi = 0
  for i in range(0,6):
    if label_uji[i] == data_uji_label[i] :
      prediksi=prediksi+1
  return prediksi

In [ ]:
prediksi_benar = func_prediksi(label_uji)
akurasi = (prediksi_benar/6)

In [ ]:
print("Banyak nilai dari Prediksi Benar ada ", prediksi_benar)

Banyak nilai dari Prediksi Benar ada  5


In [ ]:
print("Nilai Akurasi : %f atau %d Persen" % (akurasi, akurasi*100))

Nilai Akurasi : 0.833333 atau 83 Persen
